### Prebuilt react agent with search tool and memory

In [21]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import BaseMessage
from langgraph.prebuilt import create_react_agent

In [26]:
# Create the agent with memory
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatOpenAI(model='gpt-4o')
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [27]:
# Use the agent with specific thread id (memory)
config = {"configurable": {"thread_id": "user1"}}

In [29]:
# pass in config and call agent_executor.stream to interact with the agent
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in agent_executor.stream({"messages": [("user", user_input)]}, config):
        print(event)
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                print("Assistant:", value["messages"][-1].content)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n01PdLhFDRSOja2n9juaCkG0', 'function': {'arguments': '{"query": "5k races San Francisco July 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_Em8N3RfmJYzbpIk1eUonJgkh', 'function': {'arguments': '{"query": "5k races San Francisco August 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 590, 'total_tokens': 658}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-77cfa0d7-84d2-4086-8fe9-5901bb93cbf4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '5k races San Francisco July 2023'}, 'id': 'call_n01PdLhFDRSOja2n9juaCkG0', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': '5k races San Francisco August 2023'}, 'id': 'call_Em8N3RfmJYzbpI

In [25]:
# check what's stored in memory
agent_executor.get_state(config).values['messages']

[HumanMessage(content="Hi I'm Rick", id='787d088e-478e-4daa-914f-f87ec9939d66'),
 AIMessage(content='Hello, Rick! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 82, 'total_tokens': 94}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba196581-ebb1-46f8-b3db-ee84ddf2de83-0', usage_metadata={'input_tokens': 82, 'output_tokens': 12, 'total_tokens': 94}),
 HumanMessage(content="What's the weather in San Francisco?", id='b8399e78-e901-483c-b0c4-d2c895b0a5cd'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eKTatrp1cgTd17zCSOhus4UK', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 108, 'total_tokens': 130}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b